In [103]:
import numpy as np
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [104]:
data = pd.read_csv('day.csv')
data.head(10)


,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600
5,6,2011-01-06,1,0,1,0,4,1,1,0.204348,0.233209,0.518261,0.089565,88,1518,1606
6,7,2011-01-07,1,0,1,0,5,1,2,0.196522,0.208839,0.498696,0.168726,148,1362,1510
7,8,2011-01-08,1,0,1,0,6,0,2,0.165000,0.162254,0.535833,0.266804,68,891,959
8,9,2011-01-09,1,0,1,0,0,0,1,0.138333,0.116175,0.434167,0.361950,54,768,822
9,10,2011-01-10,1,0,1,0,1,1,1,0.150833,0.150888,0.482917,0.223267,41,1280,1321


Scaling the data set

In [105]:
proccessed_data = data[:]

proccessed_data['casual'] = proccessed_data['casual'].div(100)
proccessed_data['registered'] = proccessed_data['registered'].div(1000)
proccessed_data['cnt'] = proccessed_data['cnt'].div(1000)
proccessed_data.drop(['instant','dteday'],axis=1,inplace = True)


proccessed_data.head(10)

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,3.31,0.654,0.985
1,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,1.31,0.670,0.801
2,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,1.20,1.229,1.349
3,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,1.08,1.454,1.562
4,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,0.82,1.518,1.600
5,1,0,1,0,4,1,1,0.204348,0.233209,0.518261,0.089565,0.88,1.518,1.606
6,1,0,1,0,5,1,2,0.196522,0.208839,0.498696,0.168726,1.48,1.362,1.510
7,1,0,1,0,6,0,2,0.165000,0.162254,0.535833,0.266804,0.68,0.891,0.959
8,1,0,1,0,0,0,1,0.138333,0.116175,0.434167,0.361950,0.54,0.768,0.822
9,1,0,1,0,1,1,1,0.150833,0.150888,0.482917,0.223267,0.41,1.280,1.321


In [106]:
sample = np.random.choice(proccessed_data.index, size=int(len(proccessed_data)*0.9), replace=False)
train_data, test_data = proccessed_data.iloc[sample], proccessed_data.drop(sample)
features = train_data.drop('cnt', axis=1)
targets = train_data['cnt']
features_test = test_data.drop('cnt', axis=1)
targets_test = test_data['cnt']
epochs=1000

In [108]:
class NeuralNetwork(object):
    def __init__(self, input_nodes, hidden_nodes = 2, output_nodes = 1, learning_rate = 0.1):
        # Set number of nodes in input, hidden and output layers.
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        # Initialize weights
        self.weights_input_to_hidden = np.random.normal(0.0, self.input_nodes**-0.5, 
                                       (self.input_nodes, self.hidden_nodes))

        self.weights_hidden_to_output = np.random.normal(0.0, self.hidden_nodes**-0.5, 
                                       (self.hidden_nodes, self.output_nodes))
        self.lr = learning_rate
        
        
        #### TODO: Set self.activation_function to your implemented sigmoid function ####
        #
        # Note: in Python, you can define a function with a lambda expression,
        # as shown below.
        self.activation_function = lambda x : 1 / (1 + np.exp(-x))  # Replace 0 with your sigmoid calculation.
        
        self.sigmoid_prime = lambda output : output * (1 - output)
        
        def error_formula(y, final_outputs):
             return - y*np.log(final_output) - (1 - y) * np.log(1-final_outputs)
            
        def error_term_formula(x, y, output):
             return (y - output)*self.sigmoid_prime(x)
        
        ### If the lambda code above is not something you're familiar with,
        # You can uncomment out the following three lines and put your 
        # implementation there instead.
        #
        #def sigmoid(x):
        #    return 0  # Replace 0 with your sigmoid calculation here
        #self.activation_function = sigmoid
                    

    def train(self, features, epochs,targets):
        ''' Train the network on batch of features and targets. 
        
            Arguments
            ---------
            
            features: 2D array, each row is one data record, each column is a feature
            targets: 1D array of target values
        
        '''
        last_loss= None
        n_records = features.shape[0]
        delta_weights_i_h = np.zeros(self.weights_input_to_hidden.shape)
        delta_weights_h_o = np.zeros(self.weights_hidden_to_output.shape)
        
        for e in range(epochs):
            for X, y in zip(features.values, targets):
                final_outputs, hidden_outputs = self.forward_pass_train(X)  # Implement the forward pass function below
                # Implement the backproagation function below
                delta_weights_i_h, delta_weights_h_o = self.backpropagation(final_outputs, hidden_outputs, X, y, 
                                                                        delta_weights_i_h, delta_weights_h_o)
                self.update_weights(delta_weights_i_h, delta_weights_h_o, n_records)


                def forward_pass_train(self, X, weights_input_to_hidden,weights_hidden_to_output):
                        ''' Implement forward pass here 
         
                            Arguments
                        ---------
                        X: features batch

                        '''
                        #### Implement the forward pass here ####
                        ### Forward pass ###
                        # TODO: Hidden layer - Replace these values with your calculations.
                        hidden_inputs = np.dot(x, self.weights_input_to_hidden)# signals into hidden layer
                        hidden_outputs = self.activation_function(hidden_inputs)
                        #self.activation_function(hidden_inputs.dot(self.weights_hidden_to_output)) 
                        # signals from hidden layer

                        # TODO: Output layer - Replace these values with your calculations.
                        final_inputs = np.dot(hidden_outputs, self.weights_hidden_to_output) # signals into final output layer
                        final_outputs = self.activation_function(final_inputs) # signals from final output layer
        
                        return final_outputs, hidden_outputs

                def backpropagation(self, final_outputs, hidden_outputs, X, y,weights_input_to_hidden, delta_weights_h_o):
                        ''' Implement backpropagation
         
                        Arguments
                        ---------
                        final_outputs: output from forward pass
                        y: target (i.e. label) batch
                        delta_weights_i_h: change in weights from input to hidden layers
                        delta_weights_h_o: change in weights from hidden to output layers

                        '''
                        #### Implement the backward pass here ####
                        ### Backward pass ###

                        # TODO: Output error - Replace this value with your calculations.
                        error = error_formular(y,final_outputs)# Output layer error is the difference between desired target and actual output.

                        # TODO: Calculate the hidden layer's contribution to the error
                        hidden_error = output_error_term.dot(self.weights_input_to_hidden.T)

                        # TODO: Backpropagated error terms - Replace these values with your calculations.
                        output_error_term = error_term_formula(x, y, output)

                        hidden_error_term = hidden_error

                        # Weight step (input to hidden)
                        delta_weights_i_h += hidden_error_term*X

                        self.layer_0.T.dot(layer_1_delta) 

                        # Weight step (hidden to output)
                        delta_weights_h_o += output_error_term*hidden_outputs

                        return delta_weights_i_h, delta_weights_h_o

                def update_weights(self, delta_weights_i_h, delta_weights_h_o, n_records):
                            ''' Update weights on gradient descent step
         
                            Arguments
                            ---------
                            delta_weights_i_h: change in weights from input to hidden layers
                            delta_weights_h_o: change in weights from hidden to output layers
                            n_records: number of records

                            '''
                            self.weights_hidden_to_output += self.lr * delta_weights_h_o/n_records
                            #learnrate * del_w / n_records
                            #layer_1.T.dot(layer_2_delta) * self.learning_rate# update hidden-to-output weights with gradient descent step
                            self.weights_input_to_hidden += self.lr * delta_weights_i_h/n_records# update input-to-hidden weights with gradient descent step
            
                if e % (epochs / 10) == 0:
                    final_outputs = self.activation_function(final_inputs)
                    #out = sigmoid(np.dot(features, weights))
                    loss = np.mean((final_outputs - targets) ** 2)
                    print("Epoch:", e)
                    if last_loss and last_loss < loss:
                        print("Train loss: ", loss, "  WARNING - Loss Increasing")
                    else:
                        print("Train loss: ", loss)
                    last_loss = loss
                    print("=========")
            print("Finished training!")
    
    def run(self, features):
        ''' Run a forward pass through the network with input features 
        
            Arguments
            ---------
            features: 1D array of feature values
        '''
        
        #### Implement the forward pass here ####
        # TODO: Hidden layer - replace these values with the appropriate calculations.
        hidden_inputs = np.dot(x, self.weights_input_to_hidden)# signals into hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)# signals from hidden layer
        # TODO: Output layer - Replace these values with the appropriate calculations.
        final_inputs = np.dot(hidden_outputs, self.weights_hidden_to_output) # signals into final output layer
        final_outputs = self.activation_function(final_inputs) # signals from final output layer
        
        return final_outputs
